In [28]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [62]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


market_cap = df_numfirm.multiply(df_avgsize, axis=0)
market_cap = market_cap.loc[df_48ind.index]


df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)


# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_cap)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

market_weights = market_cap.div(market_cap.sum(axis=1), axis=0)


standardized_market_caps = standardized_market_caps.loc[standardized_mom.index]
standardized_BtoM = standardized_BtoM.loc[standardized_mom.index]
standardized_mom = standardized_mom.loc[standardized_mom.index]
market_weights = market_weights.loc[standardized_mom.index]
df_48ind = df_48ind.loc[standardized_mom.index]

df4 = df_48ind.shift(-1).dropna()

standardized_market_caps = standardized_market_caps.loc[df4.index]
standardized_BtoM = standardized_BtoM.loc[df4.index]
standardized_mom = standardized_mom.loc[df4.index]
market_weights = market_weights.loc[df4.index]
df_48ind = df_48ind.loc[df4.index]

def maximize_utility(MC, BM, MOM, r, T, N, gamma):
    
    def u(x, gamma):
        return (1+x) ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
        wi_t = market_weights
        return -(1/T) * np.sum(u(np.sum((wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)) * (r),axis=1), gamma))


    # Initialisation des coefficients beta
    initial_beta = np.array([0.3, 0.3, 0.3])

    # Optimisation des coefficients beta
    result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

    # Résultats
    optimal_beta = result.x

    return optimal_beta

In [45]:
df4

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,-0.81,5.57,5.84,6.76,3.50,12.52,0.53,13.27,4.60,8.28,...,6.66,8.39,4.32,11.78,1.00,6.63,10.58,9.67,10.37,0.12
1970-07-01,-0.31,0.37,-1.71,-2.71,0.65,11.41,3.23,6.50,4.22,7.25,...,7.75,8.44,6.61,5.11,7.03,6.39,1.59,5.65,5.65,14.45
1970-08-01,14.45,5.00,4.03,4.47,0.51,6.22,19.18,7.29,6.38,11.96,...,0.19,8.96,16.36,6.81,16.07,-0.40,8.72,14.86,8.70,5.75
1970-09-01,-3.56,-0.02,4.31,-1.02,10.35,-4.35,-3.74,-2.91,-0.16,-3.68,...,-2.62,-4.95,-7.58,-0.87,-3.95,-5.22,-9.97,-9.39,-7.39,-19.30
1970-10-01,-1.80,5.60,6.68,-0.84,10.05,6.52,11.30,3.76,4.89,2.47,...,1.36,3.65,-1.05,7.77,4.16,4.75,6.04,7.36,7.11,-8.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.84,-4.28,-6.43,-5.13,-2.43,-7.38,-11.17,-6.19,-6.81,-4.41,...,-2.57,-5.15,-2.69,-6.16,-4.45,-3.92,2.57,-10.45,-3.49,-2.72
2023-09-01,-5.83,-4.98,-0.39,-4.34,-4.00,-14.31,2.77,-2.05,-0.05,4.21,...,-3.46,-6.31,-0.90,0.74,-1.71,-2.89,3.49,-9.05,-6.31,-1.77
2023-10-01,-2.29,4.26,4.79,3.90,4.76,5.66,11.08,7.90,4.97,10.63,...,10.43,12.31,7.94,6.96,7.67,13.16,3.93,9.95,13.51,6.64


In [30]:
def rolling_utility(MC, window_size):
    
    for i in range(len(MC)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
    optimal_betas = []
    for i in range(len(MC) - T):
        optimal_betas.append(maximize_utility(standardized_market_caps, standardized_BtoM, standardized_mom, df_48ind, df_48ind.shape[0], df_48ind.shape[1], 5))
    return optimal_betas

In [63]:
standardized_market_caps = standardized_market_caps.loc[:'1973-12-01']
standardized_BtoM = standardized_BtoM.loc[:'1973-12-01']
standardized_mom = standardized_mom.loc[:'1973-12-01']
market_weights = market_weights.loc[:'1973-12-01']
df_48ind = df_48ind.loc[:'1973-12-01']

In [49]:
df4.shape, standardized_market_caps.shape, standardized_BtoM.shape, standardized_mom.shape, df_48ind.shape

((643, 48), (43, 48), (43, 48), (43, 48), (43, 48))

In [64]:
maximize_utility(standardized_market_caps, standardized_BtoM, standardized_mom, df_48ind, df_48ind.shape[0], df_48ind.shape[1], 5)

array([-26.63342257,   3.91638805, -11.42845577])

In [32]:
#-(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)) * (r), gamma))

MC = standardized_market_caps
BM = standardized_BtoM
MOM = standardized_mom
r = df_48ind.shift(-1).dropna()
T = df_48ind.shape[0]
N = df_48ind.shape[1]
gamma = 5

wi_t = market_weights
beta = [0.5, 0.5, 0.5]

def u(x, gamma):
    return x ** (1 - gamma) / (1 - gamma)


-(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM) * (r),axis=1), gamma))

inf

In [ ]:
#Define the CRRA utility function
def CRRA_utility(rp):
    return (1 + rp) ** (1 - gamma) / (1 - gamma)

gamma = 5

# Define the objective function to be maximized
def objective(theta):
    rp = [sum((mkt_weights.iloc[t, i] + (1/N) * sum(theta[j] * characteristics[j].iloc[t, i] for j in range(M))) * returns.iloc[t, i] for i in range(N)) for t in range(T)]
    utility = sum(CRRA_utility(r) for r in rp)
    return -utility / (T)